# カード問題

## 問
10枚のカードに，それぞれ1から10までの番号が書いてある。
これらのグループを二つのグループA,Bに分けて，グループAのカード番号の総和を36,グルーブBの番号のカードの積が360となるようなグループ分けを行いたい。

In [1]:
from typing import NewType,List, Sequence,Tuple
from gaga.cga.component import Base, Codon, Individual,Population
from random import randrange



class Affiliation(Base):
    groups=("A","B")
    def __init__(self, group: str) -> None:
        if group not in self.groups:
            raise ValueError
        super().__init__(id=None, type=None, letter=group)
        self.group=self.letter

class Card(Codon):
    def __init__(self, id: int, value: int, a: Affiliation) -> None:
        super().__init__(id=id, type=str(value), basis=[a])
        self.value=self.type
        self.group=self.basis[0].letter

class Grouping(Individual):
    def __init__(self, id: int, cards: List[Card], score: float):
        super().__init__(id=id, gene=cards, score=score)
        self.cards=self.gene
    def display(self):
        for card in self.cards:
            print(card.value,card.group)

In [6]:
class Creator:
    def giveAffilation(self):
        groups=("A","B")
        return Affiliation("A" if randrange(0,len(groups))==0 else "B")
    def createCards(self):
        numbers=(1,2,3,4,5,6,7,8,9,10)
        return [Card(i,numbers[i],self.giveAffilation()) for i in range(len(numbers))]
    def createGroupings(self,size):    
        return [Grouping(i,self.createCards(),None) for i in range(size)]
    def createPopulation(self,popSize):
        return Population(popSize,0,self.createGroupings(popSize))

In [7]:
god=Creator()
god.createPopulation(popSize=10)